# StackOverflow answer classifier

In [1]:
import re
import sys
from datetime import datetime
from html.parser import HTMLParser

import stackexchange
import numpy as np
import pandas as pd
from stackexchange import Site, StackOverflow, Sort, DESC, ASC

import credentials

In [2]:
class HTML2String(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self.strict = False
        self.convert_charrefs= True
        self.text = []
    def handle_data(self, d):
        self.text.append(d)
    def get_data(self):
        return ''.join(self.text)

def html_to_string(html):
    s = HTML2String()
    s.feed(html)
    return s.get_data()

def code_line_count(html):
    count_lines = 0
    codes = re.findall(r'<code>.*?</code>', html, re.DOTALL)
    for code in codes:
        text = html_to_string(code)
        lines = text.split('\n')
        lines = list(filter(lambda x: len(x) > 0, lines))
        count_lines += len(lines)
    return count_lines

##Preparing data

In [12]:
qs_columns = ['id', 'author_id', 'date', 'title', 'text', 'score', 'views', 'url']
as_columns = ['id', 'author_id', 'question_id', 'date', 'text', 'code_lines', 'score', 'accepted', 'url']
us_columns = ['id', 'name', 'total_questions', 'unanswered_questions', 'total_answers', 'accepted_answers', 'reputation', 'badge_total', 'url']
qa_columns = ['id', 'text', 'q_score', 'a_score', 'qer_reputation', 'aer_reputation', 'qer_percent_answered_questions', 'aer_percent_accepted_answers', 'aer_badge_total', 'a_code_lenth', 'is_qa']

questions_df = pd.DataFrame(columns=qs_columns)
answers_df = pd.DataFrame(columns=as_columns)
users_df = pd.DataFrame(columns=us_columns)
qas_df = pd.DataFrame(columns=qa_columns)

In [16]:
so = stackexchange.Site(stackexchange.StackOverflow, app_key=credentials.KEY)
# so.impose_throttling = True
# so.throttle_stop = False
so.be_inclusive()

In [5]:
from_date = datetime.strptime('2014-01-01', '%Y-%m-%d')
# to_date = datetime.strptime('2014-12-31', '%Y-%m-%d')
to_date = datetime.strptime('2014-01-02', '%Y-%m-%d')

from_date_ts = int(from_date.timestamp())
to_date_ts = int(to_date.timestamp())

In [6]:
def count_unanswered_questions(user):
    count = 0
    unanswered_questions = user.unanswered_questions.fetch()
    count += len(unanswered_questions)
    while unanswered_questions.has_more:
        unanswered_questions = unanswered_questions.fetch_next()
        count += len(unanswered_questions)
    return count

def count_accepted_answers(user):
    count = 0
    answers = user.answers.fetch()
    accepted_answers = list(filter(lambda x: x.accepted, answers))
    count += len(accepted_answers)
    while answers.has_more:
        print('x')
        answers = answers.fetch_next()
        accepted_answers = list(filter(lambda x: x.accepted, answers))
        count += len(accepted_answers)
    return count

In [7]:
def question_to_df(question):
    print('a')
    q_id = question.id
    author_id = question.owner_id
    date = question.creation_date
    title = question.title
    text = html_to_string(question.body)
    score = question.score
    views = question.view_count
    url = question.url

    info = [q_id, author_id, date, title, text, score, views, url]
    q_df = pd.DataFrame([info], columns=qs_columns)
    return q_df

def answer_to_df(answer):
    print('b')
    a_id = answer.id
    author_id = answer.owner_id
    question_id = answer.question_id
    date = answer.creation_date
    text = html_to_string(answer.body)
    code_lines = code_line_count(answer.body)
    score = answer.score
    accepted = answer.accepted
    url = answer.url

    info = [a_id, author_id, question_id, date, text, code_lines, score, accepted, url]
    a_df = pd.DataFrame([info], columns=as_columns)
    return a_df

def user_to_df(user):
    print('c')
    u_id = user.id
    name = user.display_name
    total_questions = so.questions(user_id=user.id).total
    unanswered_questions = count_unanswered_questions(user)
    total_answers = so.answers(user_id=user.id).total
    accepted_answers = count_accepted_answers(user)
    reputation = user.reputation.real
    badge_total = user.badge_total
    url = user.url

    info = [u_id, name, total_questions, unanswered_questions, total_answers, accepted_answers, reputation, badge_total, url]
    u_df = pd.DataFrame([info], columns=us_columns)
    return u_df

In [17]:
questions = so.questions(fromdate=from_date_ts, todate=to_date_ts, sort=Sort.Creation, order=ASC, pagesize=100)
while questions.has_more:
    print(1)
    for question in questions:
        print(2)
        q_df = question_to_df(question)
        questions_df = questions_df.append(q_df, ignore_index=True)
        
        author_id = question.owner_id
        if author_id not in users_df.id.values:
            print(3)
            qer = so.user(author_id)
            u_df = user_to_df(qer)
            users_df = users_df.append(u_df, ignore_index=True)
        
        answers = question.answers
        for answer in answers:
            print(4)
            a_df = answer_to_df(answer)
            answers_df = answers_df.append(a_df, ignore_index=True)
            
            author_id = answer.owner_id
            if author_id not in users_df.id.values:
                print(5)
                aer = so.user(author_id)
                u_df = user_to_df(aer)
                users_df = users_df.append(u_df, ignore_index=True)
    
        print(6)
    print(7)
    questions = questions.fetch_next()

1
2
a
4
b
6
2
a
4
b
4
b
6
2
a
4
b
4
b
6
2
a
4
b
5
c
x
4
b
4
b
5
c
6
2
a
3
c
4
b
5
c
6
2
a
3
c
4
b
5
c
x
4
b
5
c
x
x
x
x
x
x
x
x
6
2
a
3
c
4
b
5
c


BadStatusLine: ''

In [18]:
questions_df

,id,author_id,date,title,text,score,views,url
0,20865681,3126248,2014-01-01 00:00:42,Chrome crashes when exporting file via Filesys...,Im trying to run in-browser encryption applica...,3,360,http://stackoverflow.com/questions/20865681
1,20865682,1961216,2014-01-01 00:00:43,sort struct array in c++,I have an array of a Struct and trying to sort...,0,1369,http://stackoverflow.com/questions/20865682
2,20865685,3150635,2014-01-01 00:01:11,unsupported operand types for python classes,"I'm trying to make a mock pokemon game, but ev...",0,120,http://stackoverflow.com/questions/20865685
3,20865686,554807,2014-01-01 00:01:19,"Play framework ""find"" (Finder) results missing...",Executive summary:\n\nWhen I get an object bac...,3,1364,http://stackoverflow.com/questions/20865686
4,20865681,3126248,2014-01-01 00:00:42,Chrome crashes when exporting file via Filesys...,Im trying to run in-browser encryption applica...,3,360,http://stackoverflow.com/questions/20865681
5,20865682,1961216,2014-01-01 00:00:43,sort struct array in c++,I have an array of a Struct and trying to sort...,0,1369,http://stackoverflow.com/questions/20865682
6,20865685,3150635,2014-01-01 00:01:11,unsupported operand types for python classes,"I'm trying to make a mock pokemon game, but ev...",0,120,http://stackoverflow.com/questions/20865685
7,20865686,554807,2014-01-01 00:01:19,"Play framework ""find"" (Finder) results missing...",Executive summary:\n\nWhen I get an object bac...,3,1364,http://stackoverflow.com/questions/20865686
8,20865688,247794,2014-01-01 00:01:41,Is it possible to set up Amazon EC2 auto-scali...,I need to have a different auto-scaling policy...,1,165,http://stackoverflow.com/questions/20865688
9,20865689,3127791,2014-01-01 00:01:51,What's the logic of this error?,While declaring DATABASE_VERSION I declared a...,0,56,http://stackoverflow.com/questions/20865689


In [19]:
users_df

,id,name,total_questions,unanswered_questions,total_answers,accepted_answers,reputation,badge_total,url
0,3126248,Second Last,2,1,0,0,21,2,http://stackoverflow.com/users/3126248
1,2183827,Deepsy,81,23,42,23,1004,39,http://stackoverflow.com/users/2183827
2,1961216,J .A,9,4,6,0,30,8,http://stackoverflow.com/users/1961216
3,2231098,CodeDreamer,1,0,68,45,1188,20,http://stackoverflow.com/users/2231098
4,2307626,Kamran Khan,5,4,20,2,545,16,http://stackoverflow.com/users/2307626
5,3150635,user3150635,17,2,0,0,43,5,http://stackoverflow.com/users/3150635
6,3044692,Torrinos,0,0,3,0,21,2,http://stackoverflow.com/users/3044692
7,263399,norlesh,5,1,50,18,857,13,http://stackoverflow.com/users/263399
8,554807,GreenAsJade,59,8,170,204,3217,60,http://stackoverflow.com/users/554807
9,2722959,SpartanElite,11,7,32,5,442,11,http://stackoverflow.com/users/2722959


In [20]:
answers_df

,id,author_id,question_id,date,text,code_lines,score,accepted,url
0,20865938,2183827,20865681,2014-01-01 00:49:50,With a little research I found out that 1.99MB...,18,2,False,http://stackoverflow.com/questions/20865681/20...
1,20866049,2231098,20865682,2014-01-01 01:10:39,This code just can be one method among many go...,5,5,False,http://stackoverflow.com/questions/20865682/20...
2,20866117,2307626,20865682,2014-01-01 01:23:35,#include <iostream>\n#include <algorithm>\n\nu...,56,1,False,http://stackoverflow.com/questions/20865682/20...
3,20865722,3044692,20865685,2014-01-01 00:09:01,I guess you want to pass a property of the cla...,1,2,False,http://stackoverflow.com/questions/20865685/20...
4,20865731,263399,20865685,2014-01-01 00:11:40,"with enemyUnit.getHit(tackle,unit) your passin...",2,1,False,http://stackoverflow.com/questions/20865685/20...
5,20908782,2722959,20865686,2014-01-03 11:56:28,What is happening is that you are not letting ...,23,3,False,http://stackoverflow.com/questions/20865686/20...
6,20865938,2183827,20865681,2014-01-01 00:49:50,With a little research I found out that 1.99MB...,18,2,False,http://stackoverflow.com/questions/20865681/20...
7,20866049,2231098,20865682,2014-01-01 01:10:39,This code just can be one method among many go...,5,5,False,http://stackoverflow.com/questions/20865682/20...
8,20866117,2307626,20865682,2014-01-01 01:23:35,#include <iostream>\n#include <algorithm>\n\nu...,56,1,False,http://stackoverflow.com/questions/20865682/20...
9,20865722,3044692,20865685,2014-01-01 00:09:01,I guess you want to pass a property of the cla...,1,2,False,http://stackoverflow.com/questions/20865685/20...


In [21]:
w=so.user(22656)

In [25]:
anss=w.answers.fetch()

In [26]:
len(anss)

30

In [27]:
list(filter(lambda x: x.accepted, anss))

StackExchangeError: 502 [throttle_violation]: Violation of backoff parameter

In [62]:
uq.pagesize

0

In [274]:
questions[0].answers

[<Answer 20865938 @ 7f1a48be1b70>]

In [275]:
t=so.tag('php')

In [276]:
q.answers

[<Answer 20865938 @ 7f1a4aaea908>]

In [277]:
q.tags

['javascript', 'google-chrome', 'encryption', 'html5-filesystem']

In [278]:
question = so.questions(sort=Sort.Votes, order=DESC)[0]

In [280]:
comments=question.comments

In [28]:
u=q.owner

In [158]:
v=users[0]

In [25]:
w=so.user(22656)

In [26]:
w.

<User 'Jon Skeet' (22656) @ 7f8f90959ac8>

In [63]:
users=so.users()

In [69]:
count_unanswered_questions(users[1])

1

In [27]:
w.tags.fetch()

StackExchangeError: 502 [throttle_violation]: Violation of backoff parameter

In [283]:
tt=u.tags

In [287]:
tt.fetch()

StackExchangeError: 404 [no_method]: no method found with this name

In [272]:
wt=w.tags

In [29]:
wt.fetch().

SyntaxError: invalid syntax (<ipython-input-29-4799122e29be>, line 1)

In [30]:

wt=wt.fetch_next().

SyntaxError: invalid syntax (<ipython-input-30-02b7525c8a5c>, line 2)

In [233]:
ttt=so.tags(user_id=22656)

In [235]:
ttt.build_info

(<stackexchange.site.Site at 0x7f1a68763320>,
 'tags',
 stackexchange.models.Tag,
 'tags',
 {'body': 'true',
  'comments': 'true',
  'site': 'stackoverflow.com',
  'user_id': 22656})

In [217]:
len(wt)

30

In [212]:
w.reputation

778965

In [184]:
tt=w.top_answer_tags.fetch()[0]

In [37]:
w.

StackExchangeError: 502 [throttle_violation]: Violation of backoff parameter

In [34]:
w.top_answer_tags.fetch()

(<TopTag 'c#' Q:1179 A:143932>,
 <TopTag 'java' Q:0 A:84782>,
 <TopTag '.net' Q:528 A:49755>,
 <TopTag 'linq' Q:0 A:20028>,
 <TopTag 'string' Q:91 A:11850>,
 <TopTag 'generics' Q:71 A:11393>,
 <TopTag 'date' Q:0 A:8897>,
 <TopTag 'multithreading' Q:0 A:7856>,
 <TopTag 'timezone' Q:0 A:7468>,
 <TopTag 'arrays' Q:0 A:5866>,
 <TopTag 'reflection' Q:157 A:5730>,
 <TopTag 'asp.net' Q:163 A:5447>,
 <TopTag 'xml' Q:0 A:5081>,
 <TopTag 'performance' Q:204 A:4619>,
 <TopTag 'datetime' Q:0 A:4579>,
 <TopTag 'list' Q:0 A:4502>,
 <TopTag 'constructor' Q:0 A:4153>,
 <TopTag 'c#-4.0' Q:222 A:4148>,
 <TopTag 'collections' Q:0 A:4021>,
 <TopTag 'lambda' Q:0 A:3939>,
 <TopTag 'vb.net' Q:0 A:3869>,
 <TopTag 'static' Q:0 A:3467>,
 <TopTag 'visual-studio' Q:0 A:3441>,
 <TopTag 'delegates' Q:0 A:3265>,
 <TopTag 'casting' Q:0 A:3124>,
 <TopTag 'oop' Q:0 A:3065>,
 <TopTag 'inheritance' Q:0 A:2991>,
 <TopTag 'enums' Q:0 A:2948>,
 <TopTag 'floating-point' Q:0 A:2793>,
 <TopTag 'exception' Q:71 A:2727>,
 <TopTa

In [164]:
type(u)

stackexchange.models.User

In [165]:
type(v)

stackexchange.models.User

In [166]:
type(w)

stackexchange.models.User

In [169]:
qq=so.questions(user_id=22656)

In [181]:
qqq.total

34

In [180]:
qqq=so.questions(user_id=22656, answered=False)

In [176]:
len(w.questions.fetch_next())

4

In [446]:
count=0
for q in questions:
    print(q.title)
    print('-' * 80)
    if count == 10:
        break
    count += 1

Chrome crashes when exporting file via Filesystem API
--------------------------------------------------------------------------------
sort struct array in c++
--------------------------------------------------------------------------------
unsupported operand types for python classes
--------------------------------------------------------------------------------
Play framework "find" (Finder) results missing OneToMany relationship data
--------------------------------------------------------------------------------
Is it possible to set up Amazon EC2 auto-scaling for particular days of the week?
--------------------------------------------------------------------------------
What's the logic of this error?
--------------------------------------------------------------------------------
Multi-line header is not working.
--------------------------------------------------------------------------------
Codeigniter page caching: Is 24Gb of cache files too much?
---------------------------

In [441]:
len(questions)

100

In [447]:
questions.fetch()a

SyntaxError: invalid syntax (<ipython-input-447-bdbe80c41630>, line 1)

In [426]:
questions.total

2349610

In [422]:
questions[29].creation_date

datetime.datetime(2014, 1, 1, 0, 13, 32)

In [381]:
dt = datetime.strptime('1980-06-13', '%Y-%m-%d')

In [391]:
dt.timestamp()

329716800.0

In [393]:
q.creation_date.timestamp()

1340805096.0

In [448]:
qq=so.questions(todate=datetime.strptime('1980-06-13', '%Y-%m-%d').timestamp())a

SyntaxError: invalid syntax (<ipython-input-448-df33aa3b509a>, line 1)

In [354]:
df=pd.DataFrame([1,2,3])

In [357]:
df

,0
0,1
1,2
2,3


In [362]:
df.ix[1]

0    2
Name: 1, dtype: int64

In [366]:
aa=np.array([1,2])

In [368]:
aa.view()

array([1, 2])

In [3]:
me_id = 2055126
user_id = 41981
me = so.user(me_id)
user = so.user(user_id)

In [47]:
rs=user.answers.fetch()

In [48]:
rs.pagesize = 100

In [51]:
len(rs.fetch())

30

In [52]:
questions = so.questions(sort=Sort.Votes, order=DESC)

In [32]:
q=questions[0]

In [59]:
q.owner

<User 'GManNickG' (87234) @ 7f7267d31b38>

In [94]:
q.view_count

595789

In [3]:
answers=so.answers(sort=Sort.Votes, order=DESC)

NameError: name 'so' is not defined

In [2]:
a=answers[0]

NameError: name 'answers' is not defined

In [113]:
a.question_id

11227809

In [260]:
import re

In [270]:
re.findall(r'x.+?x', 'asxyxdasxzxdasda')

['xyx', 'xzx']

In [329]:
re.findall(r'<code>.*?</code>', a.body, re.DOTALL)

['<code>if (data[c] &gt;= 128)\n    sum += data[c];\n</code>',
 '<code>T = branch taken\nN = branch not taken\n\ndata[] = 0, 1, 2, 3, 4, ... 126, 127, 128, 129, 130, ... 250, 251, 252, ...\nbranch = N  N  N  N  N  ...   N    N    T    T    T  ...   T    T    T  ...\n\n       = NNNNNNNNNNNN ... NNNNNNNTTTTTTTTT ... TTTTTTTTTT  (easy to predict)\n</code>',
 '<code>data[] = 226, 185, 125, 158, 198, 144, 217, 79, 202, 118,  14, 150, 177, 182, 133, ...\nbranch =   T,   T,   N,   T,   T,   T,   T,  N,   T,   N,   N,   T,   T,   T,   N  ...\n\n       = TTNTTTTNTNNTTTN ...   (completely random - hard to predict)\n</code>',
 '<code>if (data[c] &gt;= 128)\n    sum += data[c];\n</code>',
 '<code>int t = (data[c] - 128) &gt;&gt; 31;\nsum += ~t &amp; data[c];\n</code>',
 '<code>data[]</code>',
 '<code>//  Branch - Random\nseconds = 11.777\n\n//  Branch - Sorted\nseconds = 2.352\n\n//  Branchless - Random\nseconds = 2.564\n\n//  Branchless - Sorted\nseconds = 2.587\n</code>',
 '<code>//  Branch - Ra

In [316]:
qqq='<pre><code>if (data[c] &gt;= 128)\n    sum += data[c];\n</code></pre>'

In [317]:
qqq

'<pre><code>if (data[c] &gt;= 128)\n    sum += data[c];\n</code></pre>'

In [325]:
'1  4  7  0'.split(' ')

['1', '', '4', '', '7', '', '0']

In [343]:
l=['asdasd', '', 'utyutyu']
l = filter(lambda x: len(x) > 0, l)

In [344]:
for x in l:
    print(x)

asdasd
utyutyu


In [347]:
codes = re.findall(r'<code>.*?</code>', a.body, re.DOTALL)
count_lines = 0
for code in codes:
    text = html_to_string(code)
    lines = text.split('\n')
    lines = list(filter(lambda x: len(x) > 0, lines))
    count_lines += len(lines)
print(count_lines)

34


In [323]:
a.body[:100]

'<p><strong>You are the victim of <a href="//en.wikipedia.org/wiki/Branch_predictor">branch predictio'

In [254]:
html_to_string(a.body)[:100]

'You are the victim of branch prediction fail.\n\n\n\nWhat is Branch Prediction?\n\nConsider a railroad jun'

In [7]:
users=so.users()

In [8]:
u=users[0]

In [16]:
so.impose_throttling = False
so.throttle_stop = True

In [25]:
u.answers.

(<stackexchange.site.Site at 0x7f1a68763320>,
 'users/22656/answers',
 stackexchange.models.Answer,
 'answers',
 {'body': 'true', 'comments': 'true', 'site': 'stackoverflow.com'})

In [23]:
aa=so.answers(user_id=u.id)

In [34]:
cc=so.comments(question_id=q.id)

In [36]:
cc.

(<stackexchange.models.Comment at 0x7f1a682cccf8>,
 ...)

In [26]:
aa.total

31668

In [237]:
tags=so.tags()

In [242]:
t=tags[0]

In [248]:
t.

109957

In [232]:
type(questions)

stackexchange.core.StackExchangeResultset

####Experience

In [4]:
print('StackOverflow user %d\'s experience:' % user_id)

print('Most experienced on %s.' % user.top_answer_tags.fetch()[0].tag_name)
print('Most curious about %s.' % user.top_question_tags.fetch()[0].tag_name)

total_questions = len(user.questions.fetch())
unaccepted_questions = len(user.unaccepted_questions.fetch())
accepted = total_questions - unaccepted_questions
rate = accepted / float(total_questions) * 100
print('Accept rate is %.2f%%.' % rate)

StackOverflow user 41981's experience:
Most experienced on css.
Most curious about .net.
Accept rate is 84.62%.


####Highest voted

In [5]:
print('The highest voted question on StackOverflow is:')
question = so.questions(sort=Sort.Votes, order=DESC)[0]
print('\t%8d\t%s\t%d' % (question.id, question.title, question.score))
print()
print('Look, see:', question.url)

The highest voted question on StackOverflow is:
	11227809	Why is processing a sorted array faster than an unsorted array?	10693

Look, see: http://stackoverflow.com/questions/11227809


####Question

In [6]:
#id = int(get_input("Enter a question ID: "))
id = 11227809
question = so.question(id)

print('--- %s ---' % question.title)
print(question.body)
print()
print('%d answers.' % len(question.answers))

--- Why is processing a sorted array faster than an unsorted array? ---
<p>Here is a piece of <strong>C++</strong> code that seems very peculiar. For some strange reason, sorting the data miraculously makes the code almost six times faster.</p>

<pre class="lang-cpp prettyprint-override"><code>#include &lt;algorithm&gt;
#include &lt;ctime&gt;
#include &lt;iostream&gt;

int main()
{
    // Generate data
    const unsigned arraySize = 32768;
    int data[arraySize];

    for (unsigned c = 0; c &lt; arraySize; ++c)
        data[c] = std::rand() % 256;

    // !!! With this, the next loop runs faster
    std::sort(data, data + arraySize);

    // Test
    clock_t start = clock();
    long long sum = 0;

    for (unsigned i = 0; i &lt; 100000; ++i)
    {
        // Primary loop
        for (unsigned c = 0; c &lt; arraySize; ++c)
        {
            if (data[c] &gt;= 128)
                sum += data[c];
        }
    }

    double elapsedTime = static_cast&lt;double&gt;(clock() - start) / 

####Recent questions

In [7]:
sys.stdout.write('Loading...')
sys.stdout.flush()

questions = so.recent_questions(pagesize=10, filter='_b')
print('\r #  vote ans view')

cur = 1
for question in questions[:10]:
    print('%2d %3d  %3d  %3d \t%s' % (cur, question.score, len(question.answers), question.view_count, question.title))
    cur += 1

#num = int(get_input('Question no.: '))
num = 1
qu  = questions[num - 1]
print()
print('--- TITLE: %s' % qu.title)
print('%d votes, %d answers, %d views.' % (qu.score, len(qu.answers), qu.view_count))
print('Tagged: ' + ', '.join(qu.tags))
print()
print(qu.body[:250] + ('...' if len(qu.body) > 250 else ''))

 #  vote ans view
 1  11    0  9633 	System versus user PATH environmental variable...winmerge works only if I add the path to the user PATH
 2   0    0    4 	Referencing unbounded function in Clojure REPL
 3   0    0    2 	Make bootstrap column touch the bottom of the div
 4   0    0    2 	Select to Return Multiple Rows from Single Row
 5   0    0    7 	What is the best way to incrementally build a numpy array?
 6   0    0   12 	Why is my sanitized img tags coming up empty?
 7   1    0  597 	Highcharts - Force plot lines to consume available width?
 8   0    0    2 	Strange behaviour when I trying to override protected static variable of Abstract class
 9   0    0    8 	Requesting data from Form using Jquery - have to refresh to resubmit
10   0    0   30 	Upload images PHP to MySQL and retrieve correct images to each form

--- TITLE: System versus user PATH environmental variable...winmerge works only if I add the path to the user PATH
11 votes, 0 answers, 9633 views.
Tagged: windows,

####Search

In [8]:
term = 'python'
print('Searching for %s...' % term,)
sys.stdout.flush()

qs = so.search(intitle=term)

print('\r--- questions with "%s" in title ---' % (term))

for q in qs[:10]:
    print('%8d %s' % (q.id, q.title))

Searching for python...
--- questions with "python" in title ---
30468555 List index out of range error (Python 2.7)
30468977 Why does Python String concatination work with Russian text but string.format() dose not
30469120 Python Django migrate_schemas --shared TypeError: hasattr(): attribute name must be string upon
30426935 Reassign rows and columns in Python csv
30468920 Python-linkedin Return URL?
30468371 How to convert python timestamp string to epoch?
30468593 Adding Buffer Protocol to Python Class (Not at C Level)
30468461 Finding combinations in a list (Python)
30467495 mysql installer fails to recognize python 3.4
30468902 How can I control access to a Wiifi Router using Python-Flask?


####Stats

In [9]:
stats = so.stats()

print('Total questions:\t%d' % stats.total_questions)
print('\tAnswered:\t%d' % (stats.total_questions - stats.total_unanswered))
print('\tUnanswered:\t%d' % (stats.total_unanswered))

percent = (stats.total_unanswered / float(stats.total_questions)) * 100
print('%.2f%% unanswered. (%.2f%% answered!)' % (percent, 100 - percent))

Total questions:	9470823
	Answered:	7000654
	Unanswered:	2470169
26.08% unanswered. (73.92% answered!)


In [76]:
d1 = {'a': 1, 'accepted': True}
d2 = {'a': 8, 'accepted': False}
l = [d1, d2]
list(filter(lambda x: x['accepted'], l))

[{'a': 1, 'accepted': True}]

In [84]:
ff=filter(lambda x: x['accepted'], l)

In [87]:
list(ff)

[]